# 혼공머 08-1
> 합성곱 신경망의 구성 요소

- toc:true
- branch: master
- badges: true
- comments: true
- author: 김동준
- categories : ["Python", "혼공머"]

# 주로쓰는 패키지들

In [1]:
#collapse
import numpy as np #넘파이
import pandas as pd #판다스
from plotnine import *  #플롯나인
import matplotlib.pyplot as plt #맷플랏립
import plotly.express as px #플랏리 상호작용 그래프
from IPython.display import HTML #블로그에 html로 올리려고 변환하는 패키지
import seaborn as sns # 씨본, 히스토그램 깔끔하게 그리는 패키지
#___________________________________

from sklearn.neighbors import KNeighborsClassifier # k 최근접이웃
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor  # 결정계수 
from sklearn.metrics import mean_absolute_error # 타깃과 예측의 절댓값 오차 평균을 반환
from sklearn.linear_model import LinearRegression # 선형 회귀
from sklearn.preprocessing import PolynomialFeatures #다중회귀로의 변환기
from sklearn.preprocessing import StandardScaler #규제
from sklearn.linear_model import Ridge #릿지
from sklearn.linear_model import Lasso #라쏘
from sklearn.linear_model import LogisticRegression # 로지스틱회귀
from scipy.special import expit #시그모이드함수
from scipy.special import softmax #소프트맥스함수
from sklearn.linear_model import SGDClassifier # 확률적 경사 하강법
from sklearn.tree import DecisionTreeClassifier # 트리
from sklearn.tree import plot_tree # 트리 모형
from sklearn.model_selection import cross_validate # 교차 검증
from sklearn.model_selection import StratifiedKFold # Kfold 교차 검증
from sklearn.model_selection import GridSearchCV # 그리드 서치 (하이퍼 파라미터 튜닝)
from scipy.stats import uniform, randint #랜덤 서치
from sklearn.model_selection import RandomizedSearchCV # 랜덤 서치 클래스
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 앙상블
from sklearn.ensemble import ExtraTreesClassifier # 엑스트라 트리 앙상블
from sklearn.ensemble import GradientBoostingClassifier # 그레이디언트 부스팅 앙상블

# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance # 특성중요도
from xgboost import XGBClassifier # 알고리즘을 구현한 또다른 라이브러리1
from lightgbm import LGBMClassifier # 알고리즘을 구현한 또다른 라이브러리2 , 마이크로소프트에서 구현 


# ____________________
from sklearn.cluster import KMeans # KMeans
from sklearn.decomposition import PCA # 주성분 분석

# 7장 딥러닝
from tensorflow import keras # 케라스 
import tensorflow as tf # 텐서플로

# 합성곱 convolution

> 마치 입력 데이터에 마법의 도장을 찍어서 유용한 특성만을 드러나게 하는 것으로 비유 가능

> 7장에서 사용한 밀집층에는 뉴런마다 입력 개수만큼의 가중치가 있다. 즉 모든 입력에 가중치를 곱한다.

> 인공신경망은 처음에 가중치 w1~w10과 절편 b를 랜덤하게 초기화한 다음 에포크를 반복하면서 경사 하강법 알고리즘을 사용하여 손실이 낮아지도록 최적의 가중치와 절편을 찾아간다. : 모델 훈련

> 예를 들어 밀집층에 뉴런이 3개 있다면 출력은 3개가 된다. (입력 개수에 상관없이 동일)

> ex) 784개의 픽셀을 입력받는 은닉층의 뉴런 개수가 100개면 뉴런마다 하나씩 출력도 100개가 된다.

> 합성곱은 밀집층의 계산과 조금 다르다. 입력 데이터 전체에 가중치를 적용하는 것이 아니라 일부에 가중치를 곱한다.

> 

> 가중치 $w1$~$w3$이 입력의 처음 3개 특성과 곱해져 1개의 출력을 만든다. 그다음이 중요한데, 이뉴런이 한칸 아래로 이동해 두번째부터 네번째 특성과 곱해져 새로운 출력을 만든다.

> 여기에서 중요한 것은 첫 번째 합성곱에 사용된 가중치 $w1$~$w3$과 절편 $b$가 두 번째 합성곱에도 동일하게 사용된다.

> 이렇게 한 칸씩 아래로 이동하면서 출력을 만드는 것이 합성곱.

> 여기서는 이 뉴런의 가중치가 3개이기 때문에 모두 8개의 출력이 만들어진다.

> 8개 출력 모두 같은 뉴런이며 같은 가중치 $w1$~$w3$과 절편 $b$를 사용한다.

> 밀집층의 뉴런은 입력 개수만큼 10개의 가중치를 가지고 1개의 출력을 만든다. 합성곱 층의 누런은 3개의 가중치를 가지고 8개의 출력을 만든다.

> 합성곱 층의 뉴런에 있는 가중치 개수는 정하기 나름이다. ( 또 다른 하이퍼 파라미터 )

> 입력 데이터 위를 이동하면서 같은 도장으로 하나씩 직는 것처럼 생각할 수 있다.

> 도장을 찍을 때마다 출력이 하나씩 만들어 진다.

> **합성곱 신경망(Convolution neural network , CNN)** 에서는 완전 연결 신경망(밀집층만 이용)과 달리 뉴런을 **필터** 혹은 **커널** 이라고 부른다.

> 합성곱의 장점은 1차원이 아니라 2차원 입력에도 적용할 수 있다는 것이다.

> 입력이 2차원 배열이면 필터도 2차원이어야한다. 커널의 크기를 (3x3) 으로 가정한다.

> 왼쪽 위 모서리에서부터 합성곱을 시작한다. 입력의 9개 원소와 커널의 9개 가중치를 곱한 후 1개의 출력을 만든다.

> 필터가 오른쪽으로 한칸 이동하여 합성곱을 또 수행한다. 입력의 너비가 4이므로 더 이상 오른쪽으로 한 칸 이동할 수 없다. 이럴때는 아래로 한 칸 이동한 다음 다시 왼쪽에서부터 합성곱을 수행한다. 그리고 다시 오른쪽으로 한 칸 이동한다.

> 계속 왼쪽 위에서 오른쪽 맨 아래까지 이동하면서 출력을 만든다. 계산식은 밀집층과 크게 다르지 않다. 입력과 가중치의 행과 열을 맞추어 곱셈하고 모두 더하는 게 전부다.

> 이때 5개의 출력을 필터가 입력에 놓인 위치에 맞게 2차원으로 배치한다. 즉 왼쪽 위, 오른쪽 위, 왼쪽 아래, 오른쪽 아래 모두 4개의 위치에 해당 값을 놓는다. 이렇게 출력을 2차원으로 표현하면 (4,4) 키ㅡ기의 입력을 (2,2) 크기로 압축한 느낌이다.

> 합성곱 계산을 통해 얻은 출력을 특별히 **특성 맵(feature map)** 이라고 부른다.

> 밀집층에서 여러 개의 누런을 사용하듯이 합성곱 층에서도 여러 개의 필터를 사용한다.

> (2,2)크기의 특성 맵을 쌓으면 3차원 배열이 된다.

> 밀집층에 있는 뉴런의 가중치가 모두 다르듯이 합성곱 층에 있는 필터의 가중치(커널)도 모두 다르다. $\to$ 굳이 같은 가중치를 가진 필터를 여러 개 사용할 이유가 없다.

### 합성곱 마무리

> 실제 계산은 밀집층과 동일하게 단순히 입력과 가중치를 곱하는 것이지만 2차원 형태를 유지하는 점이 다르다.

> 입력보다 훨씬 작은 크기의 커널을 사용하고 입력 위를 (왼쪽 $\to$ 오른쪽 , 위 $\to$ 아래) 이동하면서 2차원 특성 맵을 만든다.

> 2차원 구조를 사용하기 때문에 합성곱 신경망이 이미지 처리 분야에서 뛰어난 성능을 발휘한다.

# 케라스 합성곱 층

> 케라스의 층은 모두 keras.layers 패키지 아래 클래스로 구현되어 있다. (합성곱 층도 마찬가지)

> 특별히 입력 위를 (왼쪽 $\to$ 오른쪽 , 위 $\to$ 아래) 이동하는 합성곱은 Conv2D 클래스로 제공

In [2]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu')

> Conv2D 클래스의 첫 번째 매개변수는 필터의 개수이다. kernel_size 매개변수는 필터에 사용할 커널의 크기를 지정한다.

> 필터의 개수와 커널의 크기는 반드시 지정해야 하는 매개변수이다.

> 마지막으로 밀집층에서처럼 활성화 함수를 지정한다. 여기서는 렐루 함수를 선택한다.

> 특성 맵은 활성화 함수를 적용하기 후이다. 완전연결 신경망에서처럼 합성곱 신경망에서도 종종 활성화 함수를 언급하지 않는다.

> 일반적으로 특성 맵은 활성화 함수를 통과한 값을 나타낸다.

> 커널의 크기는 (3,3) , (5,5) 크기가 권장된다. (하이퍼파라미터)

> 케라스 API를 사용하면 합성곱 층을 사용하는 것이 어렵지 안다. 이전에 Dense층을 사용했던 자리에 대신 Conv2D 층을 넣으면 된다. (kernel_size와 같이 추가적인 매개변수들을 고려해야 한다.)

> 합성곱 신경망의 정의 : 일반적으로 1개 이상의 합성곱 층을 쓴 인공 신경망.

## 패딩과 스트라이드

> 앞에서 예로 들었던 합성곱 계산은 (4,4) 크기의 입력에 (3,3) 크기의 커널을 적용하여 (2,2) 크기의 특성맵을 만듦.

> (4,4) 입력과 동일한 크기의 출력을 만들려면 마치 더 큰 입력에 합성곱 하는 척을 해야한다.

> **패딩(padding)** : 입력배열의 주위를 가상의 원소로 채우는 것 (실제 입력값이 아니라서 패딩은 0으로 채운다.)

> 즉 (4,4) 크기의 입력에 0을 1개 패딩하면 (6,6)크기의 입력이 된다. 

> **세임 패딩(same padding)** : 입력과 특성 맵의 크기를 동일하게 만들기 위해 입력 주위에 0으로 패딩 하는 것. (합성곱 신경망에서는 세임 패딩이 많이 사용)

> **밸리드 패딩(valid padding)** : 패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성 맵을 만드는 경우. (밸리드 패딩은 특성 맵의 크기가 줄어들 수 밖에 없다.)

> 적절한 패딩은 이미지의 주변에 있는 정보를 잃어버리지 않도록 도와준다. 일반적인 합성곱 신경망에서는 세임 패딩이 많이 사용된다.

> 케라스 Conv2D 클래스에서는 padding 매개변수로 패딩을 지정할 수 있다. 기본값은 'valid'로 밸리드 패딩을 나타낸다. (세임 패딩을 사용하려면 'same' 으로 지정)

In [3]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same')

> **스트라이드stride** : 합성곱에서 건너뛰는 칸수. 기본값은 1

In [4]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same', strides=1)

# 풀링

> 합성곱 층에서 만든 특성 맵의 가로세로 크기를 줄이는 역할. 특성맵의 개수는 줄이지 않음.

> 